# [Multi-AI-Foundry Agents with AutoGen 0.4](https://github.com/kinfey/MultiAIAgent/blob/main/04.AzureAIAgentWithAutoGen02.ipynb)

# Constants

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
env_or_file='./../config/models_list.json'
filter_dict = {
    'endpoint': 'https://mmai-hub04-ai-servicesfvye.openai.azure.com/',
    'deployment': 'gpt-4o-2024-08-06'
}

model_name =  filter_dict["deployment"] # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup
project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


# Helper functions
Inspired by [Migration Guide for v0.2 to v0.4](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/migration-guide.html)

In [2]:
def config_list_from_json(env_or_file, filter_dict):
    import json
    with open(env_or_file, 'r') as file:
        data = json.load(file)
    
    filtered_data = [
        item for item in data
        if item.get('endpoint') == filter_dict.get('endpoint') and item.get('deployment') == filter_dict.get('deployment')
    ]    
    return filtered_data


autogen_config = config_list_from_json(env_or_file, filter_dict)[0] # we take the first combination of model and endpoint

# beaware NOT to show the API KEY
print(f'AutoGen Configuration: {autogen_config["endpoint"]}, {autogen_config["deployment"]}, {autogen_config["api_version"]}, ...') 

AutoGen Configuration: https://mmai-hub04-ai-servicesfvye.openai.azure.com/, gpt-4o-2024-08-06, 2024-10-01-preview, ...


# Model client

In [3]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=autogen_config["endpoint"],
    api_key=autogen_config["api_key"],
    model = autogen_config["model"],
    azure_deployment = autogen_config["deployment"],
    api_version=autogen_config["api_version"],
    seed = 42,
    temperature = 0.1,
)

# Create a copy of the dictionary  
data_to_print = model_client.dump_component().config.copy()

# Remove the 'api_key' from the copy  
del data_to_print['api_key']  
  
data_to_print

{'seed': 42,
 'temperature': 0.1,
 'model': 'gpt-4o',
 'azure_endpoint': 'https://mmai-hub04-ai-servicesfvye.openai.azure.com/',
 'azure_deployment': 'gpt-4o-2024-08-06',
 'api_version': '2024-10-01-preview'}

# Create AI Foundry Project Client

In [4]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

# Create wrapper functions for AI Foundry Agent

## Wrapper function for the BING AI Foundry Agent

In [5]:
async def web_ai_agent(query: str) -> str:
    from azure.ai.projects.models import BingGroundingTool # <<<<<<<<<<<<<<< SPECIFIC FOR BING SEARCH    
    
    print("This is Bing for Azure AI Agent Service...")
    
    # Retrieve the BING Connection already associated to the AI Foundry project
    bing_connection = project_client.connections.get(connection_name=os.environ["BING_CONNECTION_NAME"])
    
    # Build BingGroundingTool
    bing = BingGroundingTool(connection_id=bing_connection.id)

    # Create the Bing Agent
    agent = project_client.agents.create_agent(
        model=model_name,
        name="bing-agent",
        instructions="""
            You are a web search agent.
            Your only tool is search_tool - use it to find information.
            You make only one search call at a time.
            Once you have the results, you never do any elaboration of the obtained results.
            """,
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
        )
    print(f"Created BING agent, ID: {agent.id}")

    # Create a thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")
    
    # Add a user message to the thread
    message = project_client.agents.create_message(
        thread_id=thread.id, 
        role="user", 
        content=query, # "What is the top news today", "Quali sono i programmi TV stasera?"
    )
    print(f"Created message, ID: {message.id}")

    # Create and process assistant run in thread with tools
    run = project_client.agents.create_and_process_run\
        (thread_id=thread.id, assistant_id=agent.id)
    
    print(f"Run finished with status: {run.status}. Run id: {run.id}")
    
    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")
        result = run.last_error
    elif run.status == 'completed':    
        messages = project_client.agents.list_messages(thread_id=thread.id)
        # get the most recent message from the assistant
        last_msg = messages.get_last_text_message_by_sender("assistant")        
        if last_msg:
            result = last_msg.text.value
            print(f"\nLast Message: {result}")

        # Annotations
        print(f"Number of annotation(s): {len(last_msg.text.annotations)}")    
        a = 0
        for annotation in last_msg.text.annotations:
            a += 1
            print(f'- Annotation {a} of {len(last_msg.text.annotations)}.\n  - Text: {annotation["text"]}\n  - URL: {annotation["url_citation"]["url"]}')
    
    print (f"Deleting agent {agent.id}...")
    project_client.agents.delete_agent(agent.id)

    return result

In [6]:
# Test the wrapper function for the BING Agent

web_result = await web_ai_agent("What is the top news today?")
web_result

This is Bing for Azure AI Agent Service...
Created BING agent, ID: asst_2JLB16r9nZSQCgBNoY9Y2P10
Created thread, ID: thread_511dWo2Qo7QWrkhL5iuRmwit
Created message, ID: msg_BAAkr8j5raEFK3OewWnxWwv3
Run finished with status: RunStatus.COMPLETED. Run id: run_X9pbQAvQmtmsIDqNpSgY12c7

Last Message: Today's top news includes updates on the US with Donald Trump preparing to sign over presidential duties, and major ongoing news such as wildfires in California. Additionally, today's events are notable as they coincide with both Martin Luther King Jr. Day and Inauguration Day【5†source】【7†source】【15†source】.
Number of annotation(s): 3
- Annotation 1 of 3.
  - Text: 【5†source】
  - URL: https://www.msn.com/en-in/politics/government/world-news-live-today-january-20-2025-donald-trump-to-sign-over-200-executive-orders-on-day-one-focusing-on-border-security-energy-and-living-costs/ar-AA1xtsJJ
- Annotation 2 of 3.
  - Text: 【7†source】
  - URL: https://en.wikipedia.org/wiki/Portal:Current_events/Janua

"Today's top news includes updates on the US with Donald Trump preparing to sign over presidential duties, and major ongoing news such as wildfires in California. Additionally, today's events are notable as they coincide with both Martin Luther King Jr. Day and Inauguration Day【5†source】【7†source】【15†source】."

## Wrapper function for the CodeInterpreter AI Foundry Agent

In [7]:
async def codeinterpreter_ai_agent(action: str) -> str:
    from azure.ai.projects.models import CodeInterpreterTool # <<<<<<<<<<<<<<< SPECIFIC FOR CODE INTERPRETER
    from azure.ai.projects.models import MessageTextContent, MessageImageFileContent
    
    print("This is CodeInterpreter for Azure AI Agent Service...")
    
    # Build CodeInterpreterTool
    code_interpreter = CodeInterpreterTool()

    # Create the Code Interpreter Agent
    agent = project_client.agents.create_agent(
        model=model_name,
        name="codeinterpreter_agent",
        instructions="""
            You are a helpful agent.
            """,
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
        )
    print(f"Created CODE INTERPRETER agent, ID: {agent.id}")

    # Create a thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")
    
    # Add a user message to the thread
    message = project_client.agents.create_message(
        thread_id=thread.id, 
        role="user", 
        content=action, # How much is seventythree elevated to the power of minus 3.5?
    )
    print(f"Created message, ID: {message.id}")

    # Create and process assistant run in thread with tools
    run = project_client.agents.create_and_process_run\
        (thread_id=thread.id, assistant_id=agent.id)
    
    print(f"Run finished with status: {run.status}. Run id: {run.id}")
    
    if run.status == 'completed':    
        messages = project_client.agents.list_messages(thread_id=thread.id)
        messages_nr = len(messages.data)
        print(f"Here is the content of the last message:")
        j = 0
        for c in messages.data[0].content:
            j += 1
            if (type(c) is MessageImageFileContent):
                print(f"CONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                result = c.text.value
                print(f"CONTENT {j} (MessageTextContent) --> Text: {result}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j}: {a.text}")    
        
        print (f"\nNr. of file path annotations: {len(messages.file_path_annotations)}")
        i=0
        for file_path_annotation in messages.file_path_annotations:
            i += 1
            print(f"{i} - File annotation paths: {file_path_annotation}")
            file_name = file_path_annotation.text.split('/')[-1]
            project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
            print(f"File annoation {i} saved as file to: {os.getcwd()}\\{file_name}")
    
        # Images
        print (f"Nr. of image contents: {len(messages.image_contents)}\n")
        
        i=0
        # Generate an image file for the bar chart
        for image_content in messages.image_contents:
            i += 1
            print(f"{i} - Image content: {image_content}")
            file_name = f"{image_content.image_file.file_id}_image_content.png"
            project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
            print(f"Image content {i} file to: {os.getcwd()}\\{file_name}")
            
    elif run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Sorry, I can't proceed because the run status is {run.status}: {run.last_error}")
        result = run.last_error

    print (f"Deleting agent {agent.id}...")
    project_client.agents.delete_agent(agent.id)

    return result

In [8]:
# Test the wrapper function for the Code Interpreter Agent

# How much is seventythree elevated to the power of minus 3.5?
codeinterpreter_result = await codeinterpreter_ai_agent("""
    Could you please create a bar chart for the operating profit using 
    the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, 
    Company D: $1.8 million
    """)

codeinterpreter_result

This is CodeInterpreter for Azure AI Agent Service...
Created CODE INTERPRETER agent, ID: asst_WAQYCbzSBEhxhGovvNsjwnRw
Created thread, ID: thread_BfK6tiDTq9PbMh8gt5iQxgwY
Created message, ID: msg_eThENCIYjxZhObn9640iEy44
Run finished with status: RunStatus.COMPLETED. Run id: run_XhMniR1FYeJn2fzcYYAtpFeC
Here is the content of the last message:
CONTENT 1 (MessageImageFileContent) --> image_file id: assistant-tG16RuYzauTePJ46n0SfZ6xN
CONTENT 2 (MessageTextContent) --> Text: The bar chart for the operating profit has been created. You can download it using the link below:

[Download Operating Profit Bar Chart](sandbox:/mnt/data/operating_profit_bar_chart.png)
>>> Annotation in MessageTextContent 2: sandbox:/mnt/data/operating_profit_bar_chart.png

Nr. of file path annotations: 1
1 - File annotation paths: {'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_bar_chart.png', 'start_index': 138, 'end_index': 186, 'file_path': {'file_id': 'assistant-iEX9Yl5oK0mImEbvbNcRsXVd'}}
F

'The bar chart for the operating profit has been created. You can download it using the link below:\n\n[Download Operating Profit Bar Chart](sandbox:/mnt/data/operating_profit_bar_chart.png)'

# AUTOGEN!

# Define the corresponding AutoGen Assistant Agents

In [9]:
# ASSISTANT AGENT 1: BING SEARCH

from autogen_agentchat.agents import AssistantAgent

bingsearch_assistant = AssistantAgent(
    name="bingsearch_assistant",
    model_client=model_client,
    tools=[web_ai_agent],
    system_message="""
    You are a search expert, help me use tools to find relevant knowledge.
    Pass your result to the next agent.
    """,
)

In [10]:
# ASSISTANT AGENT 2: CODE INTERPRETER

from autogen_agentchat.agents import AssistantAgent

codeinterpreter_assistant = AssistantAgent(
    name="codeinterpreter_assistant",
    model_client=model_client,
    tools=[codeinterpreter_ai_agent],
    system_message='''
    You are a clever assistant that is able to "DO THINGS" by running python code **without asking any confirmation**.
    If you generate any files, please **ALSO** then download them locally and tell me their path.
    Pass your result to the next agent.
    ''',
)

In [11]:
# ASSISTANT AGENT 3: ITALIAN TRANSLATOR

from autogen_agentchat.agents import AssistantAgent

italiantranslator_assistant = AssistantAgent(
    name="italiantranslator_assistant",
    model_client=model_client,
    # NO TOOLS
    system_message="""
    You are an expert translator to Italian. You always translate every text to Italian.
    As soon as the translation is done, close the conversation saying 'ITALIAN TRANSLATOR IS WILLING TO TERMINATE'.
    """,
)

# Termination Condition
It's a combination of text termination and max message termination, either of which will cause the chat to terminate.

In [12]:
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(20)

# Autogen Group Chats

## Peer-to-Peer Round Robin Group Chat

In [13]:
# The group chat will alternate between the assistant and the code executor.

from autogen_agentchat.teams import RoundRobinGroupChat

group_chat = RoundRobinGroupChat([bingsearch_assistant, codeinterpreter_assistant, italiantranslator_assistant], termination_condition=termination)

stream = group_chat.run_stream(task="""
    First, find the expected min and max temperature tomorrow in Rome, Moscow and Sydney. 
    Then, create the image of a bar chart for the two temperatures of tomorrow.
    Then, download the file on my local PC.
    Translate the answer to Italian.
    """)

from autogen_agentchat.ui import Console
await Console(stream)

---------- user ----------

    First, find the expected min and max temperature tomorrow in Rome, Moscow and Sydney. 
    Then, create the image of a bar chart for the two temperatures of tomorrow.
    Then, download the file on my local PC.
    Translate the answer to Italian.
    
This is Bing for Azure AI Agent Service...
Created BING agent, ID: asst_U1ckvojg0gZKKveikYBuMohN
Created thread, ID: thread_M9eh5GXgEA11iHrtYLqmPctd
Created message, ID: msg_oZObbxmMVW2WSXJfCBtWxp3e
Run finished with status: RunStatus.COMPLETED. Run id: run_4I6esvZFoceAbdv9iwV11P2V

Last Message: The weather forecast for Rome on January 21, 2025, predicts a minimum temperature of 6°C and a maximum temperature of 12°C【5†source】.
Number of annotation(s): 1
- Annotation 1 of 1.
  - Text: 【5†source】
  - URL: https://www.meteoprog.com/weather/Rome/month/january/
Deleting agent asst_U1ckvojg0gZKKveikYBuMohN...
This is Bing for Azure AI Agent Service...
Created BING agent, ID: asst_I15XMWfJQsTVKOQ1WgVTgI81
Create

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='\n    First, find the expected min and max temperature tomorrow in Rome, Moscow and Sydney. \n    Then, create the image of a bar chart for the two temperatures of tomorrow.\n    Then, download the file on my local PC.\n    Translate the answer to Italian.\n    ', type='TextMessage'), ToolCallRequestEvent(source='bingsearch_assistant', models_usage=RequestUsage(prompt_tokens=128, completion_tokens=82), content=[FunctionCall(id='call_rVDyeudq85Tf2ZxlojnmPDjb', arguments='{"query": "expected minimum and maximum temperature in Rome tomorrow"}', name='web_ai_agent'), FunctionCall(id='call_sYhzxhleXuGC95iUbtQY0bmx', arguments='{"query": "expected minimum and maximum temperature in Moscow tomorrow"}', name='web_ai_agent'), FunctionCall(id='call_ESySYQUmfGIKciboG4IUEXh9', arguments='{"query": "expected minimum and maximum temperature in Sydney tomorrow"}', name='web_ai_agent')], type='ToolCallRequestEvent'), ToolCallE

# START Teardown

In [14]:
# Delete all agents

print(f"{len(project_client.agents.list_agents()['data'])} agent(s) will now be deleted")

i=0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"\n{i} - Agent {pca.name} has been deleted")

0 agent(s) will now be deleted


# HIC SUNT LEONES